# MACD策略

In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed

# 基础函数
import utilsJ

## 策略主体

买入信号: MACD柱向上突破0时买入。

买入仓位：每次满仓买入(做空同理)。

卖出信号：MACD柱向下突破0时卖出。

卖出仓位：清仓。

In [16]:
class MACD(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('stake', 100),

        # Indicator params
        ('ema_short', 12),
        ('ema_long', 26),
        ('macd_period', 9),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
            #    file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Initialization
        self.order = None
        self.buyprice = None
        self.sellprice = None

        # Alias
        self.dataclose = self.data.close

        # Indicators
        ## MACD
        self.macdhisto = btind.MACDHisto(self.dataclose, 
                                         period_me1=self.p.ema_short,
                                         period_me2=self.p.ema_long,
                                         period_signal=self.p.macd_period)
                    
        # Signals
        self.buy_s = bt.And(self.macdhisto.histo > 0, self.macdhisto.histo(-1) < 0)
        self.sell_s = bt.And(self.macdhisto.histo < 0, self.macdhisto.histo(-1) > 0)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price, order.executed.size,
                          self.broker.get_cash(), self.broker.get_value()))
                self.buyprice = order.executed.price
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price, -order.executed.size,
                         self.broker.get_cash(), self.broker.get_value()))
                self.sellprice = order.executed.price
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        
        self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
        
        if self.order:
            return
        
        print(self.sell_s[0])
        if self.buy_s[0] and self.getposition(self.data).size == 0: # Open position
            ava_pos = self.broker.get_cash() // (self.dataclose[0] * self.p.stake) * self.p.stake
            self.log('BUY CREATE, Price: %.2f, Lots: %i' % 
                     (self.dataclose[0], ava_pos))
            self.order = self.buy(size=ava_pos)
                   
        elif self.sell_s[0] and self.getposition(self.data).size > 0: # Close position
            self.log('SELL CREATE (Close), Price: %.2f, Lots: %i' % 
                     (self.dataclose[0], self.getposition(self.data).size))
            self.order = self.close()

### 回测

In [ ]:
stock_index = '002057.SZ'
startdate = dt.datetime(2021,1,1) - dt.timedelta(days=20)
enddate = dt.datetime(2021,12,31)


if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(MACD, printlog=True) 

    # Create stock Data Feed
    df = utilsJ.get_stock(stock_index, startdate, enddate)
    data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)

    # Add the Data Feed to Cerebo
    cerebro.adddata(data)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(10000)

    # Set leverage
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.plot(iplot=False)